# Devinons l'humeur d'une musique

L'objectif de ce projet est de pouvoir automatiquement déterminer si une musique est triste ou bien joyeuse. Pour cela nous allons utiliser la librairie [librosa](https://librosa.org/doc/latest/index.html). Cette dernière nous permettra de visualiser et d'obtenir des informations sur les musiques.
Nous avons téléchargé 10 musiques "tristes" et 10 musiques "joyeuses". Elles sont placées dans le dossier ./data/musics/ et commence par a pour les musiques joyeuses et b pour les musiques tristes.

In [1]:
%load_ext autoreload
%autoreload 2
data_dir = "./data/"
from utilities import *

In [2]:
import librosa

ModuleNotFoundError: No module named 'librosa'

In [ ]:
## A quoi ça ressemble ?
Notre première étape va être de visualiser nos données. Peut-être que ça nous donnera des idées de features.